In [1]:
AMD_LATITUDE = '23.0364'
AMD_LONGITUDE = '72.5467'
print('The geograpical coordinates of Ahmedabad are {}, {}.'.format(AMD_LATITUDE, AMD_LONGITUDE))


The geograpical coordinates of Ahmedabad are 23.0364, 72.5467.


In [2]:
!pip install folium
import folium

ahmedabad_map = folium.Map(location = [AMD_LATITUDE, AMD_LONGITUDE], zoom_start = 13)
folium.Marker([AMD_LATITUDE, AMD_LONGITUDE]).add_to(ahmedabad_map)
ahmedabad_map

In [3]:
FOURSQUARE_CLIENT_ID = '1QADG4ZIYO3Z2GIXGEDTS44EZKG0EMJKTM0GE3SO54E2F0AN'
FOURSQUARE_CLIENT_SECRET = '1ZC0OZ5BRAM5VSYJQXTBT33LM1ULIMBWKGON31WSY0IJD5LU'
RADIUS = 5000 # 5 Km
NO_OF_VENUES = 100
VERSION = '20200506'

In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
import requests

pd.set_option('display.max_rows', None)

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(FOURSQUARE_CLIENT_ID, 
                                                                        FOURSQUARE_CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        AMD_LATITUDE, 
                                                                        AMD_LONGITUDE, 
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, RADIUS/1000))

    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)
    # Filter the columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, filtered_columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100
    
foursquare_venues = foursquare_venues.reset_index(drop = True)
print("\nTotal {} venues fetched".format(total_venues))


Total 100 venues fetched within a total radius of 5.0 Km
Total 96 venues fetched within a total radius of 5.0 Km

Total 196 venues fetched


In [ ]:
foursquare_venues

,name,categories,lat,lng
0,K K tea stall,Tea Room,23.030198,72.540535
1,Zencafe,Café,23.036322,72.549455
2,Upper Crust,Diner,23.041185,72.548784
3,Cinepolis,Multiplex,23.039896,72.531727
4,Ahmedabad One,Shopping Mall,23.039758,72.531317
5,Amdavad ni Gufa,Art Gallery,23.036299,72.549433
6,Mint Route,Vegetarian / Vegan Restaurant,23.027645,72.544113
7,7 Violettes,Bakery,23.040112,72.547647
8,Birmies,Indian Restaurant,23.027362,72.544465
9,Dangee Dums,Dessert Shop,23.027597,72.544235


In [ ]:
headers = {'user-key': '08a6538b919c629af1ec2c1c7eb591e8'}
venues_information = []

for index, row in foursquare_venues.iterrows():
    print("Fetching data for venue: {}".format(index + 1))
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url, headers = headers).json()
    if (len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
    
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])


Fetching data for venue: 1
Fetching data for venue: 2
Fetching data for venue: 3
Fetching data for venue: 4
Fetching data for venue: 5
Fetching data for venue: 6
Fetching data for venue: 7
Fetching data for venue: 8
Fetching data for venue: 9
Fetching data for venue: 10
Fetching data for venue: 11
Fetching data for venue: 12
Fetching data for venue: 13
Fetching data for venue: 14
Fetching data for venue: 15
Fetching data for venue: 16
Fetching data for venue: 17
Fetching data for venue: 18
Fetching data for venue: 19
Fetching data for venue: 20
Fetching data for venue: 21
Fetching data for venue: 22
Fetching data for venue: 23
Fetching data for venue: 24
Fetching data for venue: 25
Fetching data for venue: 26
Fetching data for venue: 27
Fetching data for venue: 28
Fetching data for venue: 29
Fetching data for venue: 30
Fetching data for venue: 31
Fetching data for venue: 32
Fetching data for venue: 33
Fetching data for venue: 34
Fetching data for venue: 35
Fetching data for venue: 36
F

In [ ]:
ahmedabad_map = folium.Map(location = [AMD_LATITUDE, AMD_LONGITUDE], zoom_start = 13)

for name, latitude, longitude in zip(foursquare_venues['name'], foursquare_venues['lat'], foursquare_venues['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(ahmedabad_map)  

ahmedabad_map

In [ ]:
ahmedabad_map = folium.Map(location = [AMD_LATITUDE, AMD_LONGITUDE], zoom_start = 13)

for venue, address, latitude, longitude in zip(zomato_venues['venue'], zomato_venues['address'], 
                                               zomato_venues['latitude'], zomato_venues['longitude']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '#cc3535',
        fill_opacity = 0.7,
        parse_html = False).add_to(ahmedabad_map)  

ahmedabad_map

In [ ]:
foursquare_venues['lat'] = foursquare_venues['lat'].apply(lambda lat: round(float(lat), 4))
foursquare_venues['lng'] = foursquare_venues['lng'].apply(lambda lng: round(float(lng), 4))
zomato_venues['latitude'] = zomato_venues['latitude'].apply(lambda lat: round(float(lat), 4))
zomato_venues['longitude'] = zomato_venues['longitude'].apply(lambda lng: round(float(lng), 4))

In [ ]:
import pandas as pd
dataset = pd.concat([foursquare_venues, zomato_venues], axis = 1)
dataset['lat_diff'] = dataset['latitude'] - dataset['lat']
dataset['lng_diff'] = dataset['longitude'] - dataset['lng']

In [ ]:
selected_venues = dataset[(abs(dataset['lat_diff']) <= 0.0006) & (abs(dataset['lng_diff']) <= 0.0006)].reset_index(drop = True)

In [ ]:
selected_venues = selected_venues[selected_venues['rating'] != 0.0]

In [ ]:
selected_venues.reset_index(inplace=True,drop=True)
selected_venues

In [1]:
selected_venues=selected_venues[selected_venues['categories'].apply(lambda a: a not in ['Farm','Men\'s Store','Zoo','Clothing Store','Historic Site','Park','Farmers Market','Multiplex','Arcade','Bookstore','Shopping Mall'] )]
print("Total venues available: {}".format(selected_venues.shape[0]))

NameError: name 'selected_venues' is not defined